# Lab | Data Aggregation and Filtering

Objective: Use Python programming techniques to filter and order data subsets, group data by one or more variables, and use aggregation functions to obtain insights, and derive meaningful conclusions from the data.

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by performing first data cleaning, formatting, and structuring.

Start by reading the dataset, standardizing column names, dealing with null values and creating the month column. You can do so by using the functions you developed in previous labs, or by reading the clean dataset you saved in the last lab `data structuring and combining`.

Then, answer the following questions:

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

2. Analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

4. Find the median customer lifetime value by education level and gender.

5. Find the maximum, minimum, and average number of open complaints by state and policy type.

6. The marketing team wants to analyze the number of policies sold by state and month.

Present the data in a table where the months are arranged as columns and the states are arranged as rows.

7. Display a new DataFrame that contains the number of policies sold by month by state for the top 3 states with the highest number of policies sold.

*Hint:* 
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

8. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [72]:
import pandas as pd
import numpy as np


url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv"
marketing_customer_analysis = pd.read_csv(url)

marketing_customer_analysis.head(3)


,Unnamed: 0,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,...,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size,Vehicle Type
0,0,DK49336,Arizona,4809.216960,No,Basic,College,2/18/11,Employed,M,...,0.0,9,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize,NaN
1,1,KX64629,California,2228.525238,No,Basic,College,1/18/11,Unemployed,F,...,0.0,1,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize,NaN
2,2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2/10/11,Employed,M,...,0.0,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,A


In [73]:
marketing_customer_analysis.columns

Index(['Unnamed: 0', 'Customer', 'State', 'Customer Lifetime Value',
       'Response', 'Coverage', 'Education', 'Effective To Date',
       'EmploymentStatus', 'Gender', 'Income', 'Location Code',
       'Marital Status', 'Monthly Premium Auto', 'Months Since Last Claim',
       'Months Since Policy Inception', 'Number of Open Complaints',
       'Number of Policies', 'Policy Type', 'Policy', 'Renew Offer Type',
       'Sales Channel', 'Total Claim Amount', 'Vehicle Class', 'Vehicle Size',
       'Vehicle Type'],
      dtype='object')

In [74]:
#cleaning columns
def cleaning_column_names(insurance_company):
    insurance_company = insurance_company.rename(columns = {'EmploymentStatus':'Employment St'})
    return insurance_company
marketing_customer_analysis = cleaning_column_names(marketing_customer_analysis)
marketing_customer_analysis.head()

,Unnamed: 0,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,Employment St,Gender,...,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size,Vehicle Type
0,0,DK49336,Arizona,4809.216960,No,Basic,College,2/18/11,Employed,M,...,0.0,9,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize,NaN
1,1,KX64629,California,2228.525238,No,Basic,College,1/18/11,Unemployed,F,...,0.0,1,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize,NaN
2,2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2/10/11,Employed,M,...,0.0,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,A
3,3,XL78013,Oregon,22332.439460,Yes,Extended,College,1/11/11,Employed,M,...,0.0,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A
4,4,QA50777,Oregon,9025.067525,No,Premium,Bachelor,1/17/11,Medical Leave,F,...,NaN,7,Personal Auto,Personal L2,Offer1,Branch,707.925645,Four-Door Car,Medsize,NaN


In [75]:
marketing_customer_analysis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10910 entries, 0 to 10909
Data columns (total 26 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Unnamed: 0                     10910 non-null  int64  
 1   Customer                       10910 non-null  object 
 2   State                          10279 non-null  object 
 3   Customer Lifetime Value        10910 non-null  float64
 4   Response                       10279 non-null  object 
 5   Coverage                       10910 non-null  object 
 6   Education                      10910 non-null  object 
 7   Effective To Date              10910 non-null  object 
 8   Employment St                  10910 non-null  object 
 9   Gender                         10910 non-null  object 
 10  Income                         10910 non-null  int64  
 11  Location Code                  10910 non-null  object 
 12  Marital Status                 10910 non-null 

In [76]:
marketing_customer_analysis.isnull().sum()

Unnamed: 0                          0
Customer                            0
State                             631
Customer Lifetime Value             0
Response                          631
Coverage                            0
Education                           0
Effective To Date                   0
Employment St                       0
Gender                              0
Income                              0
Location Code                       0
Marital Status                      0
Monthly Premium Auto                0
Months Since Last Claim           633
Months Since Policy Inception       0
Number of Open Complaints         633
Number of Policies                  0
Policy Type                         0
Policy                              0
Renew Offer Type                    0
Sales Channel                       0
Total Claim Amount                  0
Vehicle Class                     622
Vehicle Size                      622
Vehicle Type                     5482
dtype: int64

In [77]:
#null values
def null_values(insurance_company):
    null_exists = False
    for column in insurance_company:
        for value in insurance_company[column].isnull():
           if(value):
               null_exists = True
    if null_exists == True:
        insurance_company = insurance_company.dropna()
    
    return insurance_company

marketing_customer_analysis = null_values(marketing_customer_analysis)


In [78]:
marketing_customer_analysis.isnull().sum()

Unnamed: 0                       0
Customer                         0
State                            0
Customer Lifetime Value          0
Response                         0
Coverage                         0
Education                        0
Effective To Date                0
Employment St                    0
Gender                           0
Income                           0
Location Code                    0
Marital Status                   0
Monthly Premium Auto             0
Months Since Last Claim          0
Months Since Policy Inception    0
Number of Open Complaints        0
Number of Policies               0
Policy Type                      0
Policy                           0
Renew Offer Type                 0
Sales Channel                    0
Total Claim Amount               0
Vehicle Class                    0
Vehicle Size                     0
Vehicle Type                     0
dtype: int64

In [79]:
marketing_customer_analysis.head()

,Unnamed: 0,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,Employment St,Gender,...,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size,Vehicle Type
2,2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2/10/11,Employed,M,...,0.0,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,A
3,3,XL78013,Oregon,22332.439460,Yes,Extended,College,1/11/11,Employed,M,...,0.0,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A
10,10,HG93801,Arizona,5154.764074,No,Extended,High School or Below,1/2/11,Employed,M,...,0.0,1,Corporate Auto,Corporate L3,Offer2,Branch,442.521087,SUV,Large,A
13,13,KR82385,California,5454.587929,No,Basic,Master,1/26/11,Employed,M,...,0.0,4,Personal Auto,Personal L3,Offer4,Call Center,331.200000,Two-Door Car,Medsize,A
16,16,FH51383,California,5326.677654,No,Basic,High School or Below,2/7/11,Employed,F,...,0.0,6,Personal Auto,Personal L3,Offer4,Call Center,300.528579,Two-Door Car,Large,A


In [80]:
def cleaning_invalid_values(row):
    if type(row['Gender']) != float:
        if row['Gender'] == 'Male':
            return 'M'
        elif row['Gender'] == 'female' or row['Gender'] == 'Femal':
            return 'F'
        else:
            return row['Gender']

def standardized_complaints(row):
    if type(row['Number of Open Complaints']) == str:
        return int(row['Number of Open Complaints'].split('/')[1])
    else:
        return row['Number of Open Complaints']

def standardized_lifetime_value(row):
    if type(row['Customer Lifetime Value']) == str:
        return float(row['Customer Lifetime Value'].strip('%'))/100
    else:
        return row['Customer Lifetime Value']


marketing_customer_analysis['Gender'] = marketing_customer_analysis.apply(cleaning_invalid_values, axis=1)
marketing_customer_analysis['Number of Open Complaints'] = marketing_customer_analysis.apply(standardized_complaints, axis=1)
marketing_customer_analysis['Customer Lifetime Value'] = marketing_customer_analysis.apply(standardized_lifetime_value, axis=1)
marketing_customer_analysis['Effective To Date'] = pd.to_datetime(marketing_customer_analysis['Effective To Date'])
marketing_customer_analysis['Month'] = marketing_customer_analysis['Effective To Date'].dt.strftime('%B')


In [81]:
marketing_customer_analysis

,Unnamed: 0,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,Employment St,Gender,...,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size,Vehicle Type,Month
2,2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2011-02-10,Employed,M,...,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,A,February
3,3,XL78013,Oregon,22332.439460,Yes,Extended,College,2011-01-11,Employed,M,...,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A,January
10,10,HG93801,Arizona,5154.764074,No,Extended,High School or Below,2011-01-02,Employed,M,...,1,Corporate Auto,Corporate L3,Offer2,Branch,442.521087,SUV,Large,A,January
13,13,KR82385,California,5454.587929,No,Basic,Master,2011-01-26,Employed,M,...,4,Personal Auto,Personal L3,Offer4,Call Center,331.200000,Two-Door Car,Medsize,A,January
16,16,FH51383,California,5326.677654,No,Basic,High School or Below,2011-02-07,Employed,F,...,6,Personal Auto,Personal L3,Offer4,Call Center,300.528579,Two-Door Car,Large,A,February
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10902,10902,PP30874,California,3579.023825,No,Extended,High School or Below,2011-01-24,Employed,F,...,1,Personal Auto,Personal L2,Offer2,Agent,655.200000,Four-Door Car,Medsize,A,January
10903,10903,SU71163,Arizona,2771.663013,No,Basic,College,2011-01-07,Employed,M,...,1,Personal Auto,Personal L2,Offer2,Branch,355.200000,Two-Door Car,Medsize,A,January
10904,10904,QI63521,Nevada,19228.463620,No,Basic,High School or Below,2011-02-24,Unemployed,M,...,2,Personal Auto,Personal L2,Offer1,Branch,897.600000,Luxury SUV,Medsize,A,February
10906,10906,KX53892,Oregon,5259.444853,No,Basic,College,2011-01-06,Employed,F,...,6,Personal Auto,Personal L3,Offer2,Branch,273.018929,Four-Door Car,Medsize,A,January


In [82]:
marketing_customer_analysis.drop(columns='Unnamed: 0', inplace=True)

In [85]:
marketing_customer_analysis.head(3)

,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,Employment St,Gender,Income,...,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size,Vehicle Type,Month
2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2011-02-10,Employed,M,22139,...,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,A,February
3,XL78013,Oregon,22332.439460,Yes,Extended,College,2011-01-11,Employed,M,49078,...,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A,January
10,HG93801,Arizona,5154.764074,No,Extended,High School or Below,2011-01-02,Employed,M,82730,...,1,Corporate Auto,Corporate L3,Offer2,Branch,442.521087,SUV,Large,A,January


** 1.Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.**

In [87]:
# Total Claim Amount > 1,000 
# Response == Yes

new_dataframe = marketing_customer_analysis[(marketing_customer_analysis['Total Claim Amount'] > 1000) & (marketing_customer_analysis['Response'] == 'Yes')]

new_dataframe.head()

,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,Employment St,Gender,Income,...,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size,Vehicle Type,Month
236,YJ16163,Oregon,11009.130490,Yes,Premium,Bachelor,2011-01-24,Employed,F,51643,...,1,Special Auto,Special L3,Offer2,Agent,1358.400000,Luxury Car,Medsize,A,January
419,GW43195,Oregon,25807.063000,Yes,Extended,College,2011-02-13,Employed,F,71210,...,2,Personal Auto,Personal L2,Offer1,Branch,1027.200000,Luxury Car,Small,A,February
442,IP94270,Arizona,13736.132500,Yes,Premium,Master,2011-02-13,Disabled,F,16181,...,8,Personal Auto,Personal L2,Offer1,Web,1261.319869,SUV,Medsize,A,February
587,FJ28407,California,5619.689084,Yes,Premium,High School or Below,2011-01-26,Unemployed,M,0,...,1,Personal Auto,Personal L1,Offer2,Web,1027.000029,SUV,Medsize,A,January
1527,TU53781,Oregon,8427.172486,Yes,Extended,Bachelor,2011-02-10,Employed,F,46503,...,1,Corporate Auto,Corporate L3,Offer1,Agent,1032.000000,Luxury SUV,Medsize,A,February


**2.Analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign.

In [89]:
# Type Response ==Yes
#AVG  Total Claim Amount by Policy Type and Policy 

avg_tca = marketing_customer_analysis[marketing_customer_analysis['Response'] == 'Yes'].groupby(["Policy Type", "Gender"]).agg({"Total Claim Amount": "mean"}).reset_index()

avg_tca['Total Claim Amount'] = avg_tca['Total Claim Amount'].round(2)

avg_tca

,Policy Type,Gender,Total Claim Amount
0,Corporate Auto,F,404.10
1,Corporate Auto,M,391.37
2,Personal Auto,F,457.06
3,Personal Auto,M,472.49
4,Special Auto,F,495.69
5,Special Auto,M,368.52


**3Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

In [91]:
#Policy, State, >500

num_policies_state = marketing_customer_analysis.groupby(["State"]).agg({"Number of Policies": "count"}).reset_index()
customers_by_state = marketing_customer_analysis.groupby(["State"]).agg({"Customer": "count"}).reset_index()
merg_data = pd.merge(num_policies_state,customers_by_state, on="State")

filter_by_state = merg_data[merg_data["Number of Policies"] > 500]

In [92]:
filter_by_state

,State,Number of Policies,Customer
0,Arizona,868,868
1,California,1552,1552
3,Oregon,1281,1281


** 4.Find the median customer lifetime value by education level and gender.

In [93]:
#Education
#Gender
#Customer Lifetime Value
marketing_customer_analysis.groupby(["Education", "Gender"]).agg({"Customer Lifetime Value": "median"}).reset_index()


,Education,Gender,Customer Lifetime Value
0,Bachelor,F,5752.485185
1,Bachelor,M,5797.377350
2,College,F,5642.596077
3,College,M,6005.847375
4,Doctor,F,5789.850900
5,Doctor,M,5843.989737
6,High School or Below,F,5978.754765
7,High School or Below,M,6081.481291
8,Master,F,5714.165863
9,Master,M,5512.342258


 **5.Find the maximum, minimum, and average number of open complaints by state and policy type.

In [95]:
marketing_customer_analysis.groupby(["State", "Policy Type"]).agg({"Number of Open Complaints": ["max", "min", "mean"]}).reset_index()

State     Policy Type Number of Open Complaints               
                                                     max  min      mean
0      Arizona  Corporate Auto                       5.0  0.0  0.456790
1      Arizona   Personal Auto                       5.0  0.0  0.413374
2      Arizona    Special Auto                       3.0  0.0  0.229167
3   California  Corporate Auto                       4.0  0.0  0.296703
4   California   Personal Auto                       5.0  0.0  0.398589
5   California    Special Auto                       4.0  0.0  0.388889
6       Nevada  Corporate Auto                       3.0  0.0  0.393617
7       Nevada   Personal Auto                       5.0  0.0  0.422492
8       Nevada    Special Auto                       3.0  0.0  0.285714
9       Oregon  Corporate Auto                       4.0  0.0  0.331984
10      Oregon   Personal Auto                       5.0  0.0  0.373444
11      Oregon    Special Auto                       5.0  0.0  0.328571
12  Washington  Corporate Auto                       4.0  0.0  0.467532
13  Washington   Personal Auto                       5.0  0.0  0.409836
14  Washington    Special Auto                       4.0  0.0  0.562500

**6.The marketing team wants to analyze the number of policies sold by state and month.


In [98]:
pivot_table = pd.pivot_table(marketing_customer_analysis, values='Number of Policies', index='State', columns='Month', aggfunc='sum', fill_value=0)
pivot_table

Month,February,January
State,,
Arizona,1297,1344
California,2158,2439
Nevada,567,666
Oregon,1885,2009
Washington,589,585


7.Display a new DataFrame that contains the number of policies sold by month by state for the top 3 states with the highest number of policies sold.
Hint:

To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.
Next, you will select the top 3 states with the highest number of policies sold.
Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.

In [99]:
#group data by state and month then count the number of policies sold for each group
policies_sold = marketing_customer_analysis.groupby(["Month", "State"], as_index=False)["Number of Policies"].sum()

#sort the data by the count of policies sold in descending order
sorted_analysis = policies_sold.sort_values(["Number of Policies"], ascending = False)

#select the top 3 states with the highest number of policies sold.
top_3_states = sorted_analysis["State"].unique()[:3]   
top_3_months = sorted_analysis["Month"].unique()[:3]
top_3_policies = sorted_analysis["Number of Policies"].unique()[:3]

top_3 = [top_3_states, top_3_months, top_3_policies]

In [100]:
top_st = pd.DataFrame(top_3)
top_st

,0,1,2
0,California,Oregon,Arizona
1,January,February,None
2,2439,2158,2009


8. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.
Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

In [101]:
melted_df = pd.melt(marketing_customer_analysis, id_vars=['Sales Channel'], value_vars=['Response'])

filtered_df = melted_df[melted_df['value'] == 'Yes']

response_rate = filtered_df.groupby('Sales Channel').size().reset_index(name='Customer Response Rate')

In [102]:
response_rate

,Sales Channel,Customer Response Rate
0,Agent,332
1,Branch,123
2,Call Center,97
3,Web,89
